- date: 2022-02-18 11:17:17
- author: Jerry Su
- slug: Feedback-Prize-Evaluating-Student-Writing
- title: Feedback-Pr?ize-Evaluating-Student-Writing
- category:
- tags: NLP

In [59]:
import pandas as pd
import os
import tqdm

## EDA

https://www.kaggle.com/robikscube/student-writing-competition-twitch-stream?scriptVersionId=83303421

In [74]:
df = pd.read_csv('/root/.cache/data/train.csv')

In [75]:
df.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [76]:
df['discourse_type'].unique()

array(['Lead', 'Position', 'Evidence', 'Claim', 'Concluding Statement',
       'Counterclaim', 'Rebuttal'], dtype=object)

In [78]:
ids = df['id'].unique()
ids.size

15594

In [ ]:
# bad case
# 2726E31ECDC6


In [108]:
an_df = df[df['id'] == '6D8814C66BB4']
an_df

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
139141,6D8814C66BB4,1.617887e+12,0.0,141.0,Sometimes when you are in a tough spot you see...,Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
139142,6D8814C66BB4,1.617887e+12,165.0,200.0,you learn more about the situation,Claim,Claim 1,33 34 35 36 37 38
139143,6D8814C66BB4,1.617887e+12,201.0,255.0,you will probably know the aftermath of your d...,Claim,Claim 2,39 40 41 42 43 44 45 46 47
139144,6D8814C66BB4,1.617887e+12,260.0,335.0,you will have your friends to support you no m...,Claim,Claim 3,49 50 51 52 53 54 55 56 57 58 59 60 61 62
139145,6D8814C66BB4,1.617887e+12,336.0,462.0,I always think the best thing to do in any str...,Position,Position 1,63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 7...
139146,6D8814C66BB4,1.617887e+12,519.0,580.0,You will learn more about what is happening in...,Claim,Claim 4,98 99 100 101 102 103 104 105 106 107 108
139147,6D8814C66BB4,1.617887e+12,581.0,1149.0,Most likely when something first happens you w...,Evidence,Evidence 1,109 110 111 112 113 114 115 116 117 118 119 12...
139148,6D8814C66BB4,1.617887e+12,1150.0,1285.0,When your talking to your friend about the sit...,Claim,Claim 5,210 211 212 213 214 215 216 217 218 219 220 22...
139149,6D8814C66BB4,1.617887e+12,1286.0,1617.0,It's always good to know whats going to happen...,Evidence,Evidence 2,234 235 236 237 238 239 240 241 242 243 244 24...
139150,6D8814C66BB4,1.617887e+12,1618.0,1695.0,After you are done making your decision your f...,Claim,Claim 6,295 296 297 298 299 300 301 302 303 304 305 30...


In [114]:
def get_instance_with_row(df, idx: int):
    row = df.loc[idx].to_dict()
    file_name = os.path.join('/root/.cache/data/train', row['id'] + '.txt')
    with open(file_name, 'r') as fp:
        text = fp.read()
    row['text'] = text
    pred_ls =  row['predictionstring'].split(' ')
    row['start_word'], row['end_word'] = int(pred_ls[0]), int(pred_ls[-1])
    row['discourse_words'] = ' '.join(text.split()[row['start_word']:row['end_word'] + 1])
    row['discourse_chars'] = text[int(row['discourse_start']):int(row['discourse_end'])]
    return row                                  

In [115]:
instance = get_instance_with_row(df, 139144)
instance

{'id': '6D8814C66BB4',
 'discourse_id': 1617887045821.0,
 'discourse_start': 260.0,
 'discourse_end': 335.0,
 'discourse_text': 'you will have your friends to support you no matter the decision you chose ',
 'discourse_type': 'Claim',
 'discourse_type_num': 'Claim 3',
 'predictionstring': '49 50 51 52 53 54 55 56 57 58 59 60 61 62',
 'text': 'Sometimes when you are in a tough spot you seek help from friends. You will ask them "what to do" and "what would you do in this situation" . When you\'re doing this you learn more about the situation, you will probably\xa0know the aftermath of your\xa0decision, and you will have your friends to support you no matter the decision you chose. I always think the best thing to do in any stressful situation is to tell someone and let them help you figure out what to do. Here\'s multiply reasons why you should always do this.\n\nYou will learn more about what is happening in the situation. Most likely when something first happens you won\'t know what to

In [20]:
from transformers import BigBirdTokenizerFast
tokenizer = BigBirdTokenizerFast.from_pretrained('allenai/longformer-large-4096')


def mapping_word_to_token(word_ids, word_start, word_end):
    token_start, token_end = -1, -1
    for idx, word_id in enumerate(word_ids):
        if word_id == word_start:
            token_start = idx
            break
    for idx, word_id in enumerate(word_ids):
        if word_id == word_end:
            token_end = idx
    return token_start, token_end

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
text = instance['text']

In [ ]:
ss.split()

### (discourse_start, discourse_end) 与 predictionstring时什么关系？

In [ ]:

def stat_relation(df):
    cnt = 0
    stat = {}
    err = []
    for idx, item in tqdm.tqdm(df.iterrows()):
        id_, discourse_id, start, end, discourse_text, type_, type_num, pred_str = item
        file_name = os.path.join('/root/.cache/data/train', id_ + '.txt')
        with open(file_name, 'r') as fp:
            text = fp.read()
        
        #print(discourse_text)
        # text using char.
        text_with_char = text[int(start):int(end)]
        
        if discourse_text != text_with_char:
            text_with_char = text[int(start):int(end) - 1]
            if discourse_text[:-1] == text_with_char:
                stat['char'] = stat.get('char', 0) + 1
            else:
                err.append(idx)
        else:
            stat['char'] = stat.get('char', 0) + 1
            
        cnt += 1
        if cnt == 6000000:
            break

    return stat
        

stat_relation(df)

## 统计predstring正确数

In [58]:
def stat_predstring(df):
    cnt = 0
    stat = {}
    err = []
    for idx, item in tqdm.tqdm(df.iterrows()):
        id_, discourse_id, start_, end_, discourse_text, type_, type_num, pred_str = item
        file_name = os.path.join('/root/.cache/data/train', id_ + '.txt')
        with open(file_name, 'r') as fp:
            text = fp.read()
        
        pred_ls =  pred_str.split(' ')
        start, end = int(pred_ls[0]), int(pred_ls[-1])
        text_pred = text.split()[start:end + 1]
        text_pred = ' '.join(text_pred)
        if discourse_text == text_pred:
            stat['word'] = stat.get('word', 0) + 1
        else:
            print('='*200)
            print(discourse_text)
            print()
            print(text_pred)
            print('='*200)
        cnt += 1
        if cnt == 50:
            break
    return stat